In [14]:
import json
import boto3
from io import BytesIO
import joblib

BUCKET_NAME = 'ml-models-niels'
MODEL_FILE_NAME = 'model_2.0.joblib'

S3 = boto3.client('s3', region_name='eu-central-1')


# def get_latest_obj(input_bucket):
#     """
#     This function gets the last modified file from an S3 bucket.
#     :param input_bucket: S3 bucket
#     :return: key of the last modified file from the S3 bucket
#     """
#     get_last_modified = lambda obj: int(obj.last_modified.strftime('%s'))
#     objs = [obj for obj in sorted(list(input_bucket.objects.all()), key=get_last_modified)]
#     return objs[-1].key

In [21]:
def load_model(key):    
    # Load model from S3 bucket
    response = S3.get_object(Bucket=BUCKET_NAME, Key=key)
    # Load pickle model
    model_str = response['Body'].read()     
    #model = joblib.load(model_str)   
    
    return model_str

In [22]:
S3

In [23]:
model = load_model(MODEL_FILE_NAME)

In [24]:
model.

b'\x80\x03csklearn.pipeline\nPipeline\nq\x00)\x81q\x01}q\x02(X\x05\x00\x00\x00stepsq\x03]q\x04(X\r\x00\x00\x00dtypeselectorq\x05cheartdisease.transformers.dtype_selector\nDTypeSelector\nq\x06)\x81q\x07}q\x08X\x06\x00\x00\x00dtypesq\tX\x06\x00\x00\x00numberq\nsb\x86q\x0bX\x1e\x00\x00\x00corrfilterhightotalcorrelationq\x0ccheartdisease.transformers.correlation_filter\nCorrFilterHighTotalCorrelation\nq\r)\x81q\x0e}q\x0f(X\t\x00\x00\x00thresholdq\x10G?\xe0\x00\x00\x00\x00\x00\x00X\x10\x00\x00\x00columns_to_drop_q\x11]q\x12X\x07\x00\x00\x00oldpeakq\x13aub\x86q\x14X\n\x00\x00\x00knnimputerq\x15csklearn.impute._knn\nKNNImputer\nq\x16)\x81q\x17}q\x18(X\x0e\x00\x00\x00missing_valuesq\x19G\x7f\xf8\x00\x00\x00\x00\x00\x00X\r\x00\x00\x00add_indicatorq\x1a\x89X\x0b\x00\x00\x00n_neighborsq\x1bK\x05X\x07\x00\x00\x00weightsq\x1cX\x07\x00\x00\x00uniformq\x1dX\x06\x00\x00\x00metricq\x1eX\r\x00\x00\x00nan_euclideanq\x1fX\x04\x00\x00\x00copyq \x88X\x0e\x00\x00\x00n_features_in_q!K\x0cX\n\x00\x00\x00indica

In [9]:
S3

In [6]:
S3.list_objects_v2(Bucket=BUCKET_NAME)

{'ResponseMetadata': {'RequestId': '24355BD8FCD1F253',
  'HostId': 'Ao2e+1d+dclonD3B5dPzGZk6vWM7dEM1fDTe5UKNVTYjJbxUYYrPNkcLag77kE3B7Lvq7/9rTts=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ao2e+1d+dclonD3B5dPzGZk6vWM7dEM1fDTe5UKNVTYjJbxUYYrPNkcLag77kE3B7Lvq7/9rTts=',
   'x-amz-request-id': '24355BD8FCD1F253',
   'date': 'Tue, 13 Oct 2020 13:50:30 GMT',
   'x-amz-bucket-region': 'eu-central-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'model_1.4.joblib',
   'LastModified': datetime.datetime(2020, 10, 2, 14, 13, 49, tzinfo=tzutc()),
   'ETag': '"732b04d39b494f15926780850ea2b7e9"',
   'Size': 667307,
   'StorageClass': 'STANDARD'},
  {'Key': 'model_1.5.joblib',
   'LastModified': datetime.datetime(2020, 10, 2, 16, 26, 36, tzinfo=tzutc()),
   'ETag': '"407ad6437f74dc700f8568fce9ecb777"',
   'Size': 666299,
   'StorageClass': 'STANDARD'},
  {'Key': 'm